In [ ]:
#| default_exp models.glssm
%load_ext autoreload
%autoreload 2

In [ ]:
# imports for this notebook only
import fastcore.test as fct

# Example gaussian models 


## Locally constant model

The locally costant model is a very basic example of a gaussian state space model. It is a univariate model with the following dynamics:

$$
\begin{align*}
    X_{t + 1} &= X_t + \varepsilon_{t + 1} & & \varepsilon_{t + 1} \sim \mathcal N(0, \sigma^2_\varepsilon), \\
    Y_t &= X_t + \eta_t && \eta_{t} \sim \mathcal N(0, \sigma^2_\eta).
\end{align*}
$$

In this model the states $X_t$ perform a discrete time, univariate, random walk and are observed with noise $\eta_t$. 

In [ ]:
# | export
import jax.numpy as jnp
from jaxtyping import Float, Array
from isssm.typing import GLSSM


def lcm(
    n: int,  # number of time steps
    x0: Float,  # initial value
    s2_x0: Float,  # initial variance
    s2_eps: Float,  # innovation variance
    s2_eta: Float,  # observation noise variance
) -> GLSSM:  # the locally constant model
    A = jnp.ones((n, 1, 1))
    B = jnp.ones((n + 1, 1, 1))

    Sigma = jnp.concatenate((s2_x0 * jnp.ones((1, 1, 1)), s2_eps * jnp.ones((n, 1, 1))))
    Omega = jnp.ones((n + 1, 1, 1)) * s2_eta

    u = jnp.zeros((n + 1, 1)).at[0].set(x0)
    v = jnp.zeros((n + 1, 1))
    return GLSSM(u, A, Sigma, v, B, Omega)

In [ ]:
n = 10
u, A, Sigma, v, B, Omega = lcm(n, 0.0, 1.0, 1.0, 1.0)

# assess that shapes are correct
fct.test_eq(u.shape, (n + 1, 1))
fct.test_eq(A.shape, (n, 1, 1))
fct.test_eq(Sigma.shape, (n + 1, 1, 1))
fct.test_eq(v.shape, (n + 1, 1))
fct.test_eq(B.shape, (n + 1, 1, 1))
fct.test_eq(Omega.shape, (n + 1, 1, 1))

## Stationary AR(1) model

States form a stationary AR(1) process with stationary distribution $\mathcal N(\mu, \tau^2)$, observed with noise 
$$
\begin{align*}
    \alpha &\in \left( -1, 1\right) \\
     \sigma^2 &= (1 - \alpha^2)\tau^2\\
    X_{t + 1} &= \mu + \alpha (X_t - \mu) + \varepsilon_{t + 1}\\
    \varepsilon_t &\sim \mathcal N(0, \sigma^2)\\
    Y_t &= X_t + \eta_t \\
    \eta_t &\sim \mathcal N(0, \omega^2)
\end{align*}
$$


In [ ]:
# | export
def ar1(
    mu: Float,  # stationary mean
    tau2: Float,  # stationary variance
    alpha,  # dampening factor
    omega2,  # observation noise
    n: int,  # number of time steps
) -> GLSSM:
    x0 = jnp.array([mu])
    A = jnp.tile(alpha * jnp.eye(1)[None], (n, 1, 1))
    B = jnp.tile(jnp.eye(1)[None], (n + 1, 1, 1))

    sigma2 = (1 - alpha**2) * tau2
    Sigma = jnp.concatenate((tau2 * jnp.ones((1, 1, 1)), sigma2 * jnp.ones((n, 1, 1))))

    Omega = omega2 * jnp.ones((n + 1, 1, 1))

    u = jnp.zeros((n + 1, 1)).at[0].set(x0)
    v = jnp.zeros((n + 1, 1))

    return GLSSM(u, A, Sigma, v, B, Omega)

In the multivariate setting, the model reads


$$
\begin{align*}
    \alpha &\in \left( -1, 1\right) \\
    \Sigma &= (1 - \alpha^2)\Tau\\
    X_{t + 1} &= \mu + \alpha (X_t - \mu) + \varepsilon_{t + 1}\\
    \varepsilon_t &\sim \mathcal N(0, \Sigma)\\
    X_{0} &\sim \mathcal N(\mu, \Tau) \\
    Y_t &= X_t + \eta_t \\
    \eta_t &\sim \mathcal N(0, \Omega),
\end{align*}
$$

where now $\Tau$ is the stationary covariance matrix.


In [ ]:
# | export
def mv_ar1(
    mu: Float[Array, "m"],  # stationary mean
    Tau: Float[Array, "m m"],  # stationary covariance
    alpha: Float,  # dampening factor
    omega2: Float,  # observation noise
    n: int,  # number of time steps
) -> GLSSM:
    x0 = mu
    (m,) = mu.shape

    A = jnp.broadcast_to(alpha * jnp.eye(m)[None], (n, m, m))
    B = jnp.broadcast_to(jnp.eye(m)[None], (n + 1, m, m))

    Sigma = jnp.concatenate(
        (Tau[None], jnp.broadcast_to((1 - alpha**2) * Tau * jnp.eye(m), (n, m, m)))
    )
    Omega = jnp.broadcast_to(omega2 * jnp.eye(m)[None], (n + 1, m, m))
    u = jnp.zeros((n + 1, m)).at[0].set(x0)
    v = jnp.zeros((n + 1, m))
    return GLSSM(u, A, Sigma, v, B, Omega)

In [ ]:
import nbdev

nbdev.nbdev_export()